In [1]:
import pandas as pd
import numpy as np
import datetime
from EvGym import config

In [2]:
pd.Timedelta(2.333, unit='h')

Timedelta('0 days 02:19:58.800000')

In [3]:
file = "data/data-SDG/0 Generated sample (AC,poisson_fit) Horizon =2015-01-01-to-2015-12-31.csv"
df_sdg = pd.read_csv(file, parse_dates = ["Date"])

In [6]:
df_sdg["starttime_parking"] = df_sdg["Date"] + pd.to_timedelta(df_sdg["Arrival"], unit='h')
df_sdg["ts_arr"] = np.floor((df_sdg["starttime_parking"] - config.starttime_min).dt.total_seconds()/ config.timestep)
df_sdg["ts_dep"] = np.ceil(df_sdg["ts_arr"] +  df_sdg["Connected_time"])
df_sdg["soc_arr"] = np.clip(config.FINAL_SOC - 0.01 -df_sdg["Energy_required"]**config.sdg_pot / config.sdg_norm**config.sdg_pot,
                             0, config.FINAL_SOC)
df_sdg["laxity"] = (df_sdg["ts_dep"] - df_sdg["ts_arr"]) - ((config.FINAL_SOC - df_sdg["soc_arr"])* config.B) / (config.alpha_c*config.eta_c)
#df_elaad["laxity"] = df_elaad["connected_time"] - (config.FINAL_SOC - df_elaad["soc_arr"]) * config.B / (config.alpha_c*config.eta_c)

In [7]:
df_sdg.describe()

,Arrival,Connected_time,Energy_required,ts_arr,ts_dep,soc_arr,laxity
count,362125.000000,362125.000000,362125.000000,362125.000000,362125.000000,362125.000000,362125.000000
mean,13.317831,6.376973,7.517034,136128.706256,136135.586262,0.629196,4.350849
std,4.738733,5.129841,3.899081,2590.072153,2589.928154,0.239065,5.284336
min,0.076923,0.000181,0.000092,131496.000000,131499.000000,0.000000,-6.198516
25%,9.220000,2.234599,4.533464,133852.000000,133857.000000,0.500328,0.556507
50%,13.593750,4.501333,7.588365,136233.000000,136238.000000,0.656859,2.914062
75%,17.041667,9.859067,9.843516,138464.000000,138467.000000,0.827048,7.618851
max,23.952381,23.998291,23.965762,140255.000000,140273.000000,0.960000,23.925787
